In [17]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)
sns.set(rc={'figure.figsize':(20, 10)})

In [18]:
import statsmodels.api as sm 
import itertools
from scipy import stats
sns.set()
#%pylab inline

In [19]:
sales = pd.read_csv('journey/data/sales_train.csv')
sales=sales.reset_index()
shops = pd.read_csv('journey/data/shops.csv')
items = pd.read_csv('journey/data/items.csv')
item_cats = pd.read_csv('journey/data/item_categories.csv')
test = pd.read_csv('journey/data/test.csv')

In [20]:
def cat_item_trans(df):
    l=[i.split("-")[0] for i in list(df.item_category_name) ] 
    l=[i.split(" ")[0] for i in l ] 

    df['cats'] = l
    from sklearn.feature_extraction import FeatureHasher
    fh = FeatureHasher(n_features=8, input_type='string')
    hashed_features = fh.fit_transform(df['cats'])
    hashed_features = hashed_features.toarray()
    return pd.concat([df[['item_category_id']], pd.DataFrame(hashed_features,columns=["category_name_"+ str(i) for i in range (8)])], axis=1)
df=cat_item_trans(item_cats)
df.head()

,item_category_id,category_name_0,category_name_1,category_name_2,category_name_3,category_name_4,category_name_5,category_name_6,category_name_7
0,0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0
1,1,-2.0,0.0,-3.0,0.0,0.0,0.0,1.0,-2.0
2,2,-2.0,0.0,-3.0,0.0,0.0,0.0,1.0,-2.0
3,3,-2.0,0.0,-3.0,0.0,0.0,0.0,1.0,-2.0
4,4,-2.0,0.0,-3.0,0.0,0.0,0.0,1.0,-2.0


In [21]:
item_to_cat_df = items.merge(df, how="inner", on="item_category_id")
item_to_cat_df.head()

,item_name,item_id,item_category_id,category_name_0,category_name_1,category_name_2,category_name_3,category_name_4,category_name_5,category_name_6,category_name_7
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,***КОРОБКА (СТЕКЛО) D,4,40,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ (UNI) ...,5,40,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


# data leakage

In [22]:
leakage=pd.merge(test,shops, how='inner', on="shop_id")

In [23]:
leakage["shop_id"].value_counts()


59    5100
16    5100
28    5100
26    5100
25    5100
24    5100
22    5100
21    5100
19    5100
18    5100
15    5100
58    5100
14    5100
12    5100
10    5100
7     5100
6     5100
5     5100
4     5100
3     5100
31    5100
34    5100
35    5100
36    5100
57    5100
56    5100
55    5100
53    5100
52    5100
50    5100
49    5100
48    5100
47    5100
46    5100
45    5100
44    5100
42    5100
41    5100
39    5100
38    5100
37    5100
2     5100
Name: shop_id, dtype: int64

In [24]:
l=[i for i in range(60) if i not in leakage["shop_id"].values]       

In [25]:
l

[0, 1, 8, 9, 11, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54]

Several shops are duplicates of each other (according to its name). Fix train set

In [26]:
# Якутск Орджоникидзе, 56
sales.loc[sales.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
sales.loc[sales.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
sales.loc[sales.shop_id == 11, 'shop_id'] = 10


In [27]:
l=[8,9, 13, 17, 20, 23, 27, 29, 30, 32, 33, 40, 43, 51, 54]

In [28]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(sales.item_id)))), len(list(set(test.item_id))), len(test)

(363, 5100, 214200)

In [29]:
no_new_item_in_test=[[list(set(test.item_id) - set(test.item_id).intersection(set(sales.item_id)))==list(set(test.item_id) - set(
    test.item_id).intersection(set(sales[sales["shop_id"]!=i].item_id))),i] for i in l]
    
no_new_item_in_test 

[[True, 8],
 [False, 9],
 [True, 13],
 [True, 17],
 [False, 20],
 [True, 23],
 [True, 27],
 [True, 29],
 [True, 30],
 [True, 32],
 [True, 33],
 [True, 40],
 [True, 43],
 [True, 51],
 [False, 54]]

* so we should keep shop_id 9 , 20 ,54

In [30]:
to_drop=sales[sales['item_cnt_day']>200]
drop=to_drop.item_id.value_counts()[to_drop.item_id.value_counts()<11].index
cnt_day_to_drop=to_drop[to_drop.item_id.isin(drop)].item_cnt_day
sales=sales[~sales.item_cnt_day.isin(cnt_day_to_drop)]#DROP THOSE ITEMS

grouped_df = sales.groupby('item_id').agg({'item_price':['min','max','mean','std']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
grouped_df = grouped_df.reset_index()
grouped_df=grouped_df[grouped_df["item_price_std"]>2500]
outlier_df=sales[sales['item_id'].isin(grouped_df['item_id'])]
to_drop=outlier_df.groupby("item_id")['item_price'].max()
sales=sales[~((sales['item_id'].isin(to_drop.index)) & (sales['item_price'].isin(to_drop)))]

In [31]:
l=[8, 23, 27, 29, 32, 33, 40]

#drop shop_id that do not exist in test set
sales=sales[~sales["shop_id"].isin(l)]



In [32]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(sales.item_id)))), len(list(set(test.item_id))), len(test)

(363, 5100, 214200)

In [33]:
sales.shape

(2753515, 7)

In [34]:
#transform test data 
#test=pd.merge(test,df, how='inner', on="shop_id")
#test= pd.merge(test,item_to_cat_df, how='inner', on="item_id")

In [35]:
#test.columns

In [36]:
#let drop ID, item_name,item_category_id from test data
#test=test.drop(columns=test.columns[[0,9,10]])

In [37]:
import gc
from tqdm import tqdm_notebook
def downcast_dtypes(df):
    
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    for i in float_cols:
        df[i] = df[i].astype(np.float32)
    for j in int_cols  : 
        df[j]   = df[j].astype(np.int32)
    
    return df


def rmse(*args):
    
    """ Funcion that calculates the root mean squared error"""
    return np.sqrt(mean_squared_error(*args))

def get_feature_matrix(sales, test, items, list_lags, date_block_threshold):
    
  
    # Create "grid" with columns
    index_cols = ['shop_id', 'item_id', 'date_block_num']

    # For every month we create a grid from all shops/items combinations from that month
    grid = [] 
    new_items = pd.DataFrame()
    cur_items_aux=np.array([])
    for block_num in sales['date_block_num'].unique():
        cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
        cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].append(pd.Series(cur_items_aux)).unique()
        cur_items_aux = cur_items[pd.Series(cur_items).isin(test.item_id)]
        grid.append(np.array(list(itertools.product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

    # Turn the grid into a dataframe
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

    # Add submission shop_id-item_id in order to test predictions
    test['date_block_num'] = 34
    grid = grid.append(test[['shop_id', 'item_id', 'date_block_num']])

    # Groupby data to get shop-item-month aggregates
    gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum','target_mean':np.mean}})
    # Fix column names
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
    # Join it to the grid
    all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

    # Same as above but with shop-month aggregates
    gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'shop__target_mean':np.mean}})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

#----------------------------------------------------------------------------------------------------------------------------    
    item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()
    sales= pd.merge(sales, item_category_mapping, how='left', on='item_id')
    all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
    all_data = downcast_dtypes(all_data)    
    
    
    # Same as above but with item_category-month aggregates
    gb = sales.groupby(['item_category_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item__target_mean':np.mean}})
    gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=['item_category_id', 'date_block_num']).fillna(0)
    
    gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'item_mean':np.mean}})
    gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

    # Downcast dtypes from 64 to 32 bit to save memory
    all_data = downcast_dtypes(all_data)
    del grid, gb 
    gc.collect()
    index_cols=index_cols+['item_category_id']
    # List of columns that we will use to create lags
    cols_to_rename = list(all_data.columns.difference(index_cols)) 

    shift_range = list_lags

    for month_shift in tqdm_notebook(shift_range):
        train_shift = all_data[index_cols + cols_to_rename].copy()
    
        train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
        foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
        train_shift = train_shift.rename(columns=foo)

        all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

    del train_shift

    # Don't use old data from year 2013
    all_data = all_data[all_data['date_block_num'] >= date_block_threshold] 

    # List of all lagged features
    fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
    # We will drop these at fitting stage
    to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

    # Category for each item

    gc.collect();
    
    return [all_data, to_drop_cols]


def clip20(x):
    return np.clip(x, 0, 20)

def clip40(x):
    return np.clip(x, 0, 20)

In [38]:
list_lags = [1, 2, 3,6,12]
date_block_threshold = 12
[all_data, to_drop_cols]  = get_feature_matrix(sales, test, items, list_lags, date_block_threshold)

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


A Jupyter Widget

In [39]:
all_data = downcast_dtypes(all_data)

In [40]:
all_data.shape

(6738186, 34)

mean_enc_item_cat = pd.DataFrame(all_data.groupby(['shop_id', 
                                                    'item_category_id']).target.agg(['mean', 'var']).reset_index())
mean_enc_item_cat.columns = ['shop_id', 'item_category_id', 'mean_enc_cat_id', 'var_enc_cat_id']
all_data = pd.merge(all_data, mean_enc_item_cat, how='left', on=['shop_id', 'item_category_id'])
del mean_enc_item_cat
all_data = downcast_dtypes(all_data)

In [41]:
all_data.tail()


,shop_id,item_id,date_block_num,target,target_mean,shop__target_mean,item_category_id,item__target_mean,item_mean,item__target_mean_lag_1,item_mean_lag_1,shop__target_mean_lag_1,target_lag_1,target_mean_lag_1,item__target_mean_lag_2,item_mean_lag_2,shop__target_mean_lag_2,target_lag_2,target_mean_lag_2,item__target_mean_lag_3,item_mean_lag_3,shop__target_mean_lag_3,target_lag_3,target_mean_lag_3,item__target_mean_lag_6,item_mean_lag_6,shop__target_mean_lag_6,target_lag_6,target_mean_lag_6,item__target_mean_lag_12,item_mean_lag_12,shop__target_mean_lag_12,target_lag_12,target_mean_lag_12
10934625,45,18454,34,0.0,0.0,0.0,55,0.0,0.0,1.022715,1.0,1.125,1.0,1.0,1.014663,1.00,1.141361,0.0,0.0,1.009725,1.0,1.085627,0.0,0.0,1.017475,1.0,1.158055,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
10934626,45,16188,34,0.0,0.0,0.0,64,0.0,0.0,1.012806,1.0,1.125,0.0,0.0,1.011364,1.00,1.141361,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
10934627,45,15757,34,0.0,0.0,0.0,55,0.0,0.0,1.022715,1.0,1.125,0.0,0.0,1.014663,1.00,1.141361,0.0,0.0,1.009725,1.0,1.085627,0.0,0.0,1.017475,1.0,1.158055,0.0,0.0,1.041970,1.0,1.268763,0.0,0.0
10934628,45,19648,34,0.0,0.0,0.0,40,0.0,0.0,1.055960,1.0,1.125,0.0,0.0,1.045658,1.00,1.141361,0.0,0.0,1.042110,1.0,1.085627,0.0,0.0,1.072826,1.0,1.158055,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
10934629,45,969,34,0.0,0.0,0.0,37,0.0,0.0,1.105034,1.0,1.125,0.0,0.0,1.063701,1.25,1.141361,0.0,0.0,1.056755,1.0,1.085627,0.0,0.0,1.039249,1.0,1.158055,0.0,0.0,1.076531,1.0,1.268763,0.0,0.0


In [42]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target', 'target_mean',
       'shop__target_mean', 'item_category_id', 'item__target_mean',
       'item_mean', 'item__target_mean_lag_1', 'item_mean_lag_1',
       'shop__target_mean_lag_1', 'target_lag_1', 'target_mean_lag_1',
       'item__target_mean_lag_2', 'item_mean_lag_2', 'shop__target_mean_lag_2',
       'target_lag_2', 'target_mean_lag_2', 'item__target_mean_lag_3',
       'item_mean_lag_3', 'shop__target_mean_lag_3', 'target_lag_3',
       'target_mean_lag_3', 'item__target_mean_lag_6', 'item_mean_lag_6',
       'shop__target_mean_lag_6', 'target_lag_6', 'target_mean_lag_6',
       'item__target_mean_lag_12', 'item_mean_lag_12',
       'shop__target_mean_lag_12', 'target_lag_12', 'target_mean_lag_12'],
      dtype='object')

### add city_code and new_item_cat

In [43]:
shops=pd.read_csv('journey/data/new_shops.csv')

In [44]:
gc.collect();

In [45]:
all_data.shape

(6738186, 34)

In [46]:
shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
shop_id      60 non-null int64
city_code    60 non-null int64
dtypes: int64(2)
memory usage: 1.0 KB


In [47]:
all_data=all_data.merge(shops,how="left",on="shop_id").drop_duplicates()

In [48]:
cats=pd.read_csv('journey/data/new_cats.csv')

In [49]:
all_data.shape

(6738186, 35)

In [50]:
cats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 11 columns):
item_category_name    84 non-null object
item_category_id      84 non-null int64
cats                  84 non-null object
аксессуар             84 non-null int64
игра                  84 non-null int64
игровой               84 non-null int64
книга                 84 non-null int64
консоль               84 non-null int64
подарок               84 non-null int64
цифра                 84 non-null int64
new_cats              84 non-null int64
dtypes: int64(9), object(2)
memory usage: 7.3+ KB


In [51]:
all_data=all_data.merge(cats[["item_category_id","new_cats"]],how="left",on="item_category_id").drop_duplicates()

In [52]:
all_data.shape

(6738186, 36)

### new time features 

In [53]:
sales["date"]=pd.DatetimeIndex(sales.date)
sales["month"]=pd.DatetimeIndex(sales.date).month
sales["year"]=pd.DatetimeIndex(sales.date).year
grouped_df=sales.groupby("item_id").agg({'date':['min']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
sales = pd.merge(sales, grouped_df, on=['item_id'])
sales["month_min"]=pd.DatetimeIndex(sales.date_min).month
sales["year_min"]=pd.DatetimeIndex(sales.date_min).year



In [54]:
gp=sales[["year_min",'item_id']].drop_duplicates()

all_data=all_data.merge(gp, on=['item_id'],how="left").fillna(2015)

gp=sales[["month_min",'item_id']].drop_duplicates()

all_data=all_data.merge(gp, on=['item_id'],how="left").fillna(11)

In [55]:
items = pd.read_csv('journey/data/items.csv')
cats=pd.read_csv('journey/data/new_cats.csv')
items=items.merge(cats[['item_category_id',"new_cats"]], how="left", on="item_category_id")
sales=sales.merge(items[['item_id',"new_cats","item_category_id"]], how="left", on="item_id")

In [56]:
grouped_df=sales.groupby(["year_min","month_min"]).agg({'item_cnt_day':{'mean_new':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=["year_min","month_min"],how="left").fillna(
    sales[sales.year_min==2015].item_cnt_day.mean())

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [57]:
grouped_df=sales.groupby(["year_min","month_min","shop_id"]).agg({'item_cnt_day':{'shop_id':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=["year_min","month_min","shop_id"],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [58]:
grouped_df=sales.groupby(["year_min","shop_id"]).agg({'item_cnt_day':{'year_shop_id':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=["year_min","shop_id"],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [59]:
grouped_df=sales.groupby(["year_min","month_min",'item_category_id']).agg({'item_cnt_day':{'cats_id':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=['item_category_id',"month_min","year_min"],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [60]:
grouped_df=sales.groupby(["year_min",'item_category_id']).agg({'item_cnt_day':{'year_cats_id':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=['item_category_id',"year_min"],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [61]:
grouped_df=sales[(sales.month==sales.month_min)&(sales.year==sales.year_min)].groupby(["shop_id"]).agg({'item_cnt_day':{'new_shop':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=["shop_id"],how="left")

grouped_df=sales[(sales.month==sales.month_min)&(sales.year==sales.year_min)].groupby(["item_category_id"]).agg({'item_cnt_day':{'new_cats':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=['item_category_id'],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [62]:
grouped_df=sales.groupby("item_id").agg({'date_block_num':['min']})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
sales = pd.merge(sales, grouped_df, on=['item_id'])
gp=sales[["date_block_num_min",'item_id']].drop_duplicates()
all_data=all_data.merge(gp, on=['item_id'],how="left").fillna(34)

In [63]:
grouped_df=sales[(sales.date_block_num==sales.date_block_num_min)].groupby(["shop_id"]).agg({'item_cnt_day':{'date_shop':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=["shop_id"],how="left")

grouped_df=sales[(sales.date_block_num==sales.date_block_num_min)].groupby(["item_category_id"]).agg({'item_cnt_day':{'date_cats':np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data = pd.merge(all_data, grouped_df, on=['item_category_id'],how="left")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [64]:
all_data.shape

(6738186, 48)

l=[]
for i in sales.item_id.unique():
    df=sales[sales.item_id==i][sales.date_block_num==sales[sales.item_id==i].date_block_num_min.unique()[0]]
    l=l+[[i,df.item_cnt_day.mean()]]

mean=pd.DataFrame(l,columns=['item_id', 'item_new_mean'])    

all_data=all_data.merge(mean,how="left",on=["item_id"]).fillna(mean.item_new_mean.mean())

grouped_df =sales.groupby(['item_id','date_block_num_min']).agg({'item_cnt_day':{"new_item":np.mean}})
grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
all_data=all_data.merge(grouped_df,how="left",on=["item_id"]).fillna(grouped_df.item_cnt_day_new_item.mean())

gp=sales.groupby(["date_block_num_nunique"]).agg({'item_cnt_day':{"item_time_target":np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=["date_block_num_nunique"])  

gp=sales.groupby(["shop_id","date_block_num_nunique"]).agg({'item_cnt_day':{"item_time_shop_target":np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=['shop_id',"date_block_num_nunique"])  


gp=sales.groupby(["item_id","date_block_num_min"]).agg({'item_cnt_day':{'item_shop_time_target':np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=["item_id","date_block_num_min"]).fillna(gp.item_shop_time_target.mean())    



gp=sales.groupby(["shop_id","date_block_num_min"]).agg({'item_cnt_day':{"item_time_shop_target":np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=['shop_id',"date_block_num_min"]).fillna(gp.item_time_shop_target.mean())


gp=sales.groupby(["item_category_id","date_block_num_nunique"]).agg({'item_cnt_day':{'cats_mean_target':np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=["item_category_id","date_block_num_nunique"])  


gp=sales.groupby(["item_category_id","date_block_num_min"]).agg({'item_cnt_day':{'cats_mean_price':np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data = pd.merge(all_data, gp, how='left', on=["item_category_id","date_block_num_min"]).fillna(gp.cats_mean_price.mean())  

all_data=downcast_dtypes(all_data)

In [65]:
#sub_data = all_data[all_data.date_block_num==34]
#all_data = all_data[all_data.date_block_num<34]
#sub_data.head()

In [66]:
to_drop_cols

['item_mean',
 'target_mean',
 'shop__target_mean',
 'target',
 'item__target_mean',
 'date_block_num']

In [67]:
#all_data.to_pickle('journey/data/new_train2.pickle')
#sub_data.to_pickle('journey/data/sub_data2.pickle')

In [68]:
#sales["date"]=pd.DatetimeIndex(sales.date)
#sales["month"]=pd.DatetimeIndex(sales.date).month
#sales["year"]=pd.DatetimeIndex(sales.date).year
#sales["day"]=pd.DatetimeIndex(sales.date).day

### add sepecial dates

In [69]:
from generate_calendar import Calendar

start_date = '2013-01-01'
end_date = '2016-02-10'
api_key = "3f654100-f7fc-45b2-9646-8725053c9954"

calendar = Calendar(start_date, end_date, include_holidays=True, holiday_api_key=api_key, country='RU')

data_frame = calendar.generate()
data_frame.head()

,date,year,month,day,weekday,weekday_name,weeknumber,is_weekend,is_business_day,is_holiday,holiday_name
0,2013-01-01,2013,1,1,2,Tuesday,1,False,False,True,Новый год Novy god
1,2013-01-02,2013,1,2,3,Wednesday,1,False,False,True,Новый год
2,2013-01-03,2013,1,3,4,Thursday,1,False,False,True,Новый год
3,2013-01-04,2013,1,4,5,Friday,1,False,False,True,Новый год
4,2013-01-05,2013,1,5,6,Saturday,1,True,False,True,Новый год


In [70]:
sales.shape

(2753515, 15)

In [71]:
#all_data = all_data.append(sub_data)

In [72]:
all_data.shape

(6738186, 48)

In [73]:
gp=data_frame.groupby(["year","month"]).agg({"is_weekend":{'':np.mean},"is_business_day":{"is_business_day":np.mean},"is_holiday":{"is_holiday":np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
sales=pd.merge(sales,gp,how='left',on=["year","month"])

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [74]:
sales.head()

,index,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,date_min,month_min,year_min,new_cats,item_category_id,date_block_num_min,is_weekend,is_business_day,is_holiday
0,0,2013-02-01,0,59,22154,999.0,1.0,2,2013,2013-01-14,1,2013,5,37,0,0.285714,0.714286,0.035714
1,3270,2013-01-23,0,24,22154,999.0,1.0,1,2013,2013-01-14,1,2013,5,37,0,0.258065,0.548387,0.258065
2,25918,2013-02-01,0,25,22154,999.0,1.0,2,2013,2013-01-14,1,2013,5,37,0,0.285714,0.714286,0.035714
3,25919,2013-03-01,0,25,22154,999.0,1.0,3,2013,2013-01-14,1,2013,5,37,0,0.322581,0.645161,0.032258
4,25920,2013-01-20,0,25,22154,999.0,1.0,1,2013,2013-01-14,1,2013,5,37,0,0.258065,0.548387,0.258065


In [75]:
for i in sales.date_block_num.unique():
    print(i,"-->",sales[sales.date_block_num==i].month.value_counts().idxmax())

0 --> 1
1 --> 2
2 --> 3
3 --> 4
7 --> 8
8 --> 9
9 --> 10
11 --> 12
30 --> 7
31 --> 8
32 --> 9
33 --> 10
4 --> 5
6 --> 7
17 --> 6
28 --> 5
5 --> 6
10 --> 11
12 --> 1
13 --> 2
14 --> 3
18 --> 7
19 --> 8
21 --> 10
15 --> 4
24 --> 1
16 --> 5
20 --> 9
22 --> 11
23 --> 12
25 --> 2
26 --> 3
27 --> 4
29 --> 6


In [76]:
(all_data.date_block_num.unique()+1) % 12

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11], dtype=int64)

In [77]:
all_data.date_block_num.unique()
all_data['month'] = (all_data['date_block_num']+1) % 12
all_data.loc[all_data.month == 0,"month"] = 12

In [78]:
all_data.columns

Index(['shop_id', 'item_id', 'date_block_num', 'target', 'target_mean',
       'shop__target_mean', 'item_category_id', 'item__target_mean',
       'item_mean', 'item__target_mean_lag_1', 'item_mean_lag_1',
       'shop__target_mean_lag_1', 'target_lag_1', 'target_mean_lag_1',
       'item__target_mean_lag_2', 'item_mean_lag_2', 'shop__target_mean_lag_2',
       'target_lag_2', 'target_mean_lag_2', 'item__target_mean_lag_3',
       'item_mean_lag_3', 'shop__target_mean_lag_3', 'target_lag_3',
       'target_mean_lag_3', 'item__target_mean_lag_6', 'item_mean_lag_6',
       'shop__target_mean_lag_6', 'target_lag_6', 'target_mean_lag_6',
       'item__target_mean_lag_12', 'item_mean_lag_12',
       'shop__target_mean_lag_12', 'target_lag_12', 'target_mean_lag_12',
       'city_code', 'new_cats', 'year_min', 'month_min',
       'item_cnt_day_mean_new', 'item_cnt_day_shop_id',
       'item_cnt_day_year_shop_id', 'item_cnt_day_cats_id',
       'item_cnt_day_year_cats_id', 'item_cnt_day_new_s

In [79]:
gp=sales.groupby(["month"]).agg({"is_weekend":{'':np.mean},"is_business_day":{"is_business_day":np.mean},"is_holiday":{"is_holiday":np.mean}})
gp.columns = [col[0] if col[-1]=='' else col[-1] for col in gp.columns.values]
all_data=pd.merge(all_data,gp,how='left',on="month")

C:\Users\Hamza SBAAI\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [80]:
all_data.shape

(6738186, 52)

## Train/test split 

In [81]:
sub_data = all_data[all_data.date_block_num==34].fillna(0)
all_data = all_data[all_data.date_block_num<34].fillna(0)
sub_data.head()

,shop_id,item_id,date_block_num,target,target_mean,shop__target_mean,item_category_id,item__target_mean,item_mean,item__target_mean_lag_1,item_mean_lag_1,shop__target_mean_lag_1,target_lag_1,target_mean_lag_1,item__target_mean_lag_2,item_mean_lag_2,shop__target_mean_lag_2,target_lag_2,target_mean_lag_2,item__target_mean_lag_3,item_mean_lag_3,shop__target_mean_lag_3,target_lag_3,target_mean_lag_3,item__target_mean_lag_6,...,target_lag_6,target_mean_lag_6,item__target_mean_lag_12,item_mean_lag_12,shop__target_mean_lag_12,target_lag_12,target_mean_lag_12,city_code,new_cats,year_min,month_min,item_cnt_day_mean_new,item_cnt_day_shop_id,item_cnt_day_year_shop_id,item_cnt_day_cats_id,item_cnt_day_year_cats_id,item_cnt_day_new_shop,item_cnt_day_new_cats,date_block_num_min,item_cnt_day_date_shop,item_cnt_day_date_cats,month,is_weekend,is_business_day,is_holiday
6523986,5,5037,34,0.0,0.0,0.0,19,0.0,0.0,1.041791,1.000000,1.090155,0.0,0.0,1.273688,1.428571,1.148265,1.0,1.0,1.126207,1.190000,1.080134,3.0,1.0,1.056495,...,1.0,1.0,1.147309,1.0,1.235043,1.0,1.0,2.0,2,2014.0,1.0,1.267267,1.121926,1.110896,1.203563,1.198509,1.127524,1.214295,20.0,1.347322,1.861216,11,0.314381,0.652286,0.033333
6523987,5,5320,34,0.0,0.0,0.0,55,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,2.0,7,2015.0,11.0,1.666667,34.000000,1.144337,34.000000,1.061678,1.127524,1.037858,34.0,1.347322,1.086957,11,0.314381,0.652286,0.033333
6523988,5,5233,34,0.0,0.0,0.0,19,0.0,0.0,1.041791,1.024390,1.090155,1.0,1.0,1.273688,1.250000,1.148265,3.0,1.0,1.126207,1.181102,1.080134,1.0,1.0,1.056495,...,3.0,1.5,0.000000,0.0,0.000000,0.0,0.0,2.0,2,2015.0,1.0,1.381513,1.144856,1.144337,1.168332,1.163236,1.127524,1.214295,27.0,1.347322,1.861216,11,0.314381,0.652286,0.033333
6523989,5,5232,34,0.0,0.0,0.0,23,0.0,0.0,1.055582,0.933333,1.090155,0.0,0.0,1.261420,1.043478,1.148265,0.0,0.0,1.093142,1.203704,1.080134,1.0,1.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,2.0,2,2015.0,1.0,1.381513,1.144856,1.144337,1.148048,1.142831,1.127524,1.116452,31.0,1.347322,1.524478,11,0.314381,0.652286,0.033333
6523990,5,5268,34,0.0,0.0,0.0,20,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,2.0,2,2015.0,11.0,1.666667,34.000000,1.144337,34.000000,1.918867,1.127524,2.020486,34.0,1.347322,2.872383,11,0.314381,0.652286,0.033333


In [ ]:
all_data.to_pickle('journey/data/new_train2.pickle')

sub_data.to_pickle('journey/data/new_test2.pickle')